# BeamNGpy Feature Overview
*BeamNG.tech only*

## Introduction
This notebook is a non-exhaustive showcase of the things you can do with BeamNGpy, focused on customization of the vehicles, the environment and the sensor suite.

To install BeamNGpy, follow the instructions listed [here](https://github.com/BeamNG/BeamNGpy#installation).
Also be sure to study the complete documentation of [BeamNGpy](https://beamngpy.readthedocs.io) and [BeamNG.tech](https://documentation.beamng.com/beamng_tech/), if you want to familiarize yourself more with the library.

Let's start!

BeamNGpy is packaged as a Python module called `beamngpy`. All the imports that we will need in this notebook are listed in the following cell:

In [1]:
import colorsys
import time

import matplotlib.pyplot as plt
import numpy as np
from beamngpy import BeamNGpy, Scenario, Vehicle
from beamngpy.sensors import (
    IMU, Camera, Damage, Electrics, Lidar, Timer, Ultrasonic)
from IPython.display import clear_output

To start BeamNG.tech using BeamNGpy, you use the constructor of the class `BeamNGpy`. The `BeamNGpy` instance allows you to control the simulation. We call `BeamNGpy.open()` to start the simulation which is connected to the Python process using a TCP socket.

#### Documentation

In [2]:
?BeamNGpy

Init signature: BeamNGpy(host, port, home=None, user=None, remote=False)
Docstring:     
The BeamNGpy class is the backbone of communication with the BeamNG
simulation and offers methods of starting, stopping, connecting to, and
controlling the state of the simulator.
Init docstring:
Instantiates a BeamNGpy instance connecting to the simulator on the
given host and port. The home directory of the simulator can be passed
to this constructor. If None is given, this class tries to read a
home path from the ``BNG_HOME`` environment variable.

Note:
    If no home path is set, this class will not work properly.

Args:
    host (str): The host to connect to
    port (int): The port to connect to
    home (str): Path to the simulator's home directory.
    user (str): Additional optional user path to set. This path can be
                used to set where custom files created during
                executions will be placed if the home folder shall not
                be touched.
    remote (b

In [3]:
?BeamNGpy.open

Signature: BeamNGpy.open(self, extensions=None, *args, launch=True, **opts)
Docstring:
Starts a BeamNG.* process, opens a server socket, and waits for the
spawned BeamNG.* process to connect. This method blocks until the
process started and is ready.

Args:
    launch (bool): Whether to launch a new process or connect to a
                   running one on the configured host/port. Defaults to
                   True.
File:      beamngpy\src\beamngpy\beamng.py
Type:      function


#### Code

In [4]:
beamng = BeamNGpy('localhost', 64256)
beamng.open()

The main menu of BeamNG.tech pops up and we are ready to set up the simulation! We will start on the `italy` map. Let's generate a `Scenario` object, a single `Vehicle` and add the vehicle to the scenario.

#### Documentation

In [5]:
?Scenario

Init signature: Scenario(level, name, path=None, **options)
Docstring:     
The scenario class contains information for setting up and executing
simulation scenarios along with methods to extract data during their
execution.
Init docstring:
Instantiates a scenario instance with the given name taking place in
the given level.

Args:
    level: Either the name of the level this scenario takes place in
           as a string or as an instance of :class:`.Level`
    name (str): The name of this scenario. Should be unique for the
                level it's taking place in to avoid file collisions.
File:           beamngpy\src\beamngpy\scenario.py
Type:           type
Subclasses:     


In [6]:
?Vehicle

Init signature: Vehicle(vid, model, port=None, **options)
Docstring:     
The vehicle class represents a vehicle of the simulation that can be
interacted with from BeamNGpy. This class offers methods to both control
the vehicle's state as well as retrieve information about it through
sensors the user can attach to the vehicle.
Init docstring:
Creates a vehicle with the given vehicle ID. The ID must be unique
within the scenario.

Args:
    vid (str): The vehicle's ID.
    model (str): Model of the vehicle.
File:           beamngpy\src\beamngpy\vehicle.py
Type:           type
Subclasses:     


In [7]:
?Scenario.add_vehicle

Signature:
Scenario.add_vehicle(
    self,
    vehicle,
    pos=(0, 0, 0),
    rot_quat=(0, 0, 0, 1),
    cling=True,
)
Docstring:
Adds a vehicle to this scenario at the given position with the given
orientation.

Args:
    pos (tuple): (x,y,z) tuple specifying the position of the vehicle.
    rot_quat (tuple, optional): (x, y, z, w) tuple specifying
                                the rotation as quaternion
File:      beamngpy\src\beamngpy\scenario.py
Type:      function


In [8]:
?Scenario.make

Signature: Scenario.make(self, bng)
Docstring:
Generates necessary files to describe the scenario in the simulation
and outputs them to the simulator.

Args:
    bng (:class:`.BeamNGpy`): The BeamNGpy instance to generate the
                              scenario for.

Raises:
    BNGError: If the scenario already has set its info .json file included.
File:      beamngpy\src\beamngpy\scenario.py
Type:      function


#### Code

In [9]:
scenario = Scenario('italy', 'beamngpy_feature_overview')

ego = Vehicle('ego', model='etk800', color='White', licence='PYTHON')
scenario.add_vehicle(ego, pos=(245.11, -906.94, 247.46),
                     rot_quat=(0.0010, 0.1242, 0.9884, -0.0872))
scenario.make(beamng)

Voilà, simple as that! To actually start the simulation, you need the following commands:

#### Documentation

In [10]:
?BeamNGpy.load_scenario

Signature: BeamNGpy.load_scenario(self, scenario)
Docstring:
Loads the given scenario in the simulation and returns once loading
is finished.

Args:
    scenario (:class:`.Scenario`): The scenario to load.
File:      beamngpy\src\beamngpy\beamng.py
Type:      function


In [11]:
?BeamNGpy.start_scenario

Signature: BeamNGpy.start_scenario(self, restrict_actions=False)
Docstring:
Starts the scenario; equivalent to clicking the "Start" button in the
game after loading a scenario. This method blocks until the countdown
to the scenario's start has finished.

Args:
    restrict_actions (bool): Whether to keep scenario restrictions,
                             such as limited menu options and controls.
                             Defaults to False.
File:      beamngpy\src\beamngpy\beamng.py
Type:      function


#### Code

In [12]:
beamng.load_scenario(scenario)
beamng.start_scenario()

The scenario is now loaded and started. Get ready for the feature showcase! We start by using the `ai_set_mode` command on the vehicle. It will drive around through the whole map, using data about the road network.

#### Documentation

In [13]:
?Vehicle.ai_set_mode

Signature: Vehicle.ai_set_mode(self, mode)
Docstring:
Sets the desired mode of the simulator's built-in AI for this vehicle.
Possible values are:

 * ``disabled``: Turn the AI off (default state)
 * ``random``: Drive from random points to random points on the map
 * ``span``: Drive along the entire road network of the map
 * ``manual``: Drive to a specific waypoint, target set separately
 * ``chase``: Chase a target vehicle, target set separately
 * ``flee``: Flee from a vehicle, target set separately
 * ``stopping``: Make the vehicle come to a halt (AI disables itself
                                                    once the vehicle
                                                    stopped.)

Note:
    Some AI methods automatically set appropriate modes, meaning a call
    to this method might be optional.

Args:
    mode (str): The AI mode to set.
File:      beamngpy\src\beamngpy\vehicle.py
Type:      function


In [14]:
?Vehicle.ai_drive_in_lane

Signature: Vehicle.ai_drive_in_lane(self, lane)
Docstring:
Sets the drive in lane flag of the AI. If True, the AI only drives
within the lane it can legally drive in.

Args:
    lane (bool): Lane flag to set.
File:      beamngpy\src\beamngpy\vehicle.py
Type:      function


#### Code

In [15]:
ego.ai_set_mode('span')
ego.ai_drive_in_lane(True)

## Customization options

With BeamNGpy, we can set and tweak various parameters of the vehicles spawned and also of the whole simulation environment. Let's see a few examples of what is possible to do. This list is not exhaustive; consult the [documentation](https://beamngpy.readthedocs.io) to see all the available options.

#### Documentation

In [16]:
?Vehicle.set_color

Signature: Vehicle.set_color(self, rgba=(1.0, 1.0, 1.0, 1.0))
Docstring:
Sets the color of this vehicle. Colour can be adjusted on the RGB
spectrum and the "shininess" of the paint.

Args:
    rgba (tuple): The new colour given as a tuple of RGBA floats, where
                  the alpha channel encodes the shininess of the paint.
File:      beamngpy\src\beamngpy\vehicle.py
Type:      function


In [17]:
?Vehicle.set_lights

Signature: Vehicle.set_lights(self, **kwargs)
Docstring:
Sets the vehicle's lights to given intensity values. The lighting
system features lights that are simply binary on/off, but also ones
where the intensity can be varied. Binary lights include:

    * `left_signal`
    * `right_signal`
    * `hazard_signal`

Non-binary lights vary between 0 for off, 1 for on, 2 for higher
intensity. For example, headlights can be turned on with 1 and set to
be more intense with 2. Non-binary lights include:

    * `headlights`
    * `fog_lights`
    * `lightbar`

Args:
    left_signal (bool): On/off state of the left signal
    right_signal (bool): On/off state of the right signal
    hazard_signal (bool): On/off state of the hazard lights
    headlights (int): Value from 0 to 2 indicating headlight intensity
    fog_lights (int): Value from 0 to 2 indicating fog light intensity
    lightbar (int): Value from 0 to 2 indicating lightbar intensity

Note:
    Not every vehicle has every type of light.

In [18]:
?BeamNGpy.set_tod

Signature: BeamNGpy.set_tod(self, tod)
Docstring:
Sets the current time of day. The time of day value is given as a float
between 0 and 1. How this value affects the lighting of the scene is
dependant on the map's TimeOfDay object.

Args:
    tod (float): Time of day beteen 0 and 1.
File:      beamngpy\src\beamngpy\beamng.py
Type:      function


In [19]:
?BeamNGpy.pause

Signature: BeamNGpy.pause(self)
Docstring:
Sends a pause request to BeamNG.*, blocking until the simulation is
paused.
File:      beamngpy\src\beamngpy\beamng.py
Type:      function


In [20]:
?BeamNGpy.resume

Signature: BeamNGpy.resume(self)
Docstring:
Sends a resume request to BeamNG.*, blocking until the simulation
is resumed.
File:      beamngpy\src\beamngpy\beamng.py
Type:      function


In [21]:
?BeamNGpy.set_gravity

Signature: BeamNGpy.set_gravity(self, gravity=-9.807)
Docstring:
Sets the strength of gravity in the simulator.

Args:
    gravity (float): The gravity value to set. The default one is
                     that of earth (-9.807)
File:      beamngpy\src\beamngpy\beamng.py
Type:      function


In [22]:
?BeamNGpy.display_gui_message

Signature: BeamNGpy.display_gui_message(self, msg)
Docstring:
Displays a toast message in the user interface of the simulator.

Args:
    msg (str): The message to display.
File:      beamngpy\src\beamngpy\beamng.py
Type:      function


### Change color

Cycles through 30 different colors and applies them to the vehicle in sequence.

In [23]:
def generate_colors(n):
    hue = 0.0
    step_val = 1.0 / n
    for _ in range(n):
        rgb = colorsys.hsv_to_rgb(hue, 1, 1)
        hue += step_val
        hue %= 1.0
        yield (*rgb, 1.0)

for color in generate_colors(30):
    ego.set_color(color)
    time.sleep(0.1)
ego.set_color((1.0, 0.5, 0.5, 1.0))

### Switch lights

Switches all lights on and then resets them to the original state.

In [24]:
ego.set_lights(left_signal=True, right_signal=True, hazard_signal=True,
               headlights=2, fog_lights=2, lightbar=2)

In [25]:
ego.set_lights(left_signal=False, right_signal=False, hazard_signal=False,
               headlights=1, fog_lights=0, lightbar=0)

### Set time of day

In [26]:
for t in range(100):
    beamng.set_tod(t / 100)
    time.sleep(0.1)

### Pause/resume simulation

In [27]:
beamng.pause()

In [28]:
beamng.resume()

### Set gravity

In [29]:
beamng.set_gravity(0)
beamng.display_gui_message('Space adventures await!')

In [30]:
beamng.set_gravity(-50)

OK, that was too much gravity for the car. Let's reset it back. We also use the `teleport_vehicle` function, which resets the vehicle's physics state (repairs it) as a side effect.

### Teleport vehicle

#### Documentation

In [31]:
?Scenario.find_waypoints

Signature: Scenario.find_waypoints(self)
Docstring:
Finds waypoints placed in the world right now.

Returns:
    A list of :class:`.ScenarioObject` containing waypoints found in
    the world.

Raises:
    BNGError: If the scenario is not currently loaded.
File:      beamngpy\src\beamngpy\scenario.py
Type:      function


In [32]:
?Vehicle.teleport

Signature: Vehicle.teleport(self, pos, rot_quat=None, reset=True)
Docstring:
Teleports the vehicle to the given position with the given
rotation.

Args:
    pos (tuple): The target position as an (x,y,z) tuple containing
                 world-space coordinates.
    rot_quat (tuple): Optional tuple (x, y, z, w) specifying vehicle
                      rotation as quaternion
    reset (bool): Specifies if the vehicle will be reset to its initial
                  state during teleport (including its velocity). 

Notes:
    The ``reset=False`` option is incompatible with setting rotation of
    the vehicle. With the current implementation, it is not possible to
    set the rotation of the vehicle and to keep its velocity during teleport.
File:      beamngpy\src\beamngpy\vehicle.py
Type:      function


In [33]:
?Vehicle.set_shift_mode

Signature: Vehicle.set_shift_mode(self, mode)
Docstring:
Sets the shifting mode of the vehicle. This changes whether or not and
how the vehicle shifts gears depending on the RPM. Available modes are:

 * ``realistic_manual``: Gears have to be shifted manually by the
                         user, including engaging the clutch.
 * ``realistic_manual_auto_clutch``: Gears have to be shifted manually
                                     by the user, without having to
                                     use the clutch.
 * ``arcade``: Gears shift up and down automatically. If the brake is
               held, the vehicle automatically shifts into reverse
               and accelerates backward until brake is released or
               throttle is engaged.
 * ``realistic_automatic``: Gears shift up automatically, but reverse
                            and parking need to be shifted to
                            manually.

Args:
    mode (str): The mode to set. Must be a string from the opt

In [34]:
?Vehicle.control

Signature: Vehicle.control(self, **options)
Docstring:
Sends a control message to the vehicle, setting vehicle inputs
accordingly. Possible values to set are:

 * ``steering``: Rotation of the steering wheel, from -1.0 to 1.0.
 * ``throttle``: Intensity of the throttle, from 0.0 to 1.0.
 * ``brake``: Intensity of the brake, from 0.0 to 1.0.
 * ``parkingbrake``: Intensity of the parkingbrake, from 0.0 to 1.0.
 * ``clutch``: Clutch level, from 0.0 to 1.0.
 * ``gear``: Gear to shift to, -1 eq backwards, 0 eq neutral, 1 to X eq nth gear

Args:
    **kwargs (dict): The input values to set.
File:      beamngpy\src\beamngpy\vehicle.py
Type:      function


In [35]:
?Vehicle.set_velocity

Signature: Vehicle.set_velocity(self, velocity, dt=1.0)
Docstring:
Sets the velocity of this vehicle. The velocity is not achieved instantly,
it is acquired gradually over the time interval set by the `dt` argument.

As the method of setting velocity uses physical forces, at high velocities
it is important to set `dt` to an appropriately high value. The default
`dt` value of 1.0 is suitable for velocities up to 30 m/s.

Args:
    velocity (float): The target velocity in m/s.
    dt (float): The time interval over which the vehicle reaches the
                target velocity. Defaults to 1.0.
File:      beamngpy\src\beamngpy\vehicle.py
Type:      function


#### Code

In [36]:
# reset gravity, disable AI
beamng.set_gravity()
ego.ai_set_mode('disabled')

In [37]:
waypoints = {w.name: w for w in scenario.find_waypoints()} # get a list of locations from the simulation

ego.teleport(waypoints['city_east_bridge1_b'].pos, rot_quat=(0.0011, 0.0315, 0.9995, -0.0096))

True

If we need to teleport a moving vehicle to some position and keep it moving, we set the `reset` argument of `Vehicle.teleport` to `False`.

In [38]:
# get the vehicle moving
ego.set_shift_mode('arcade')
ego.control(gear=1, throttle=0.1)
ego.set_velocity(10)

# wait some time
time.sleep(2.0)

# teleport the vehicle and enable AI again
ego.teleport((244.87, -926.57, 251.89), reset=False)
ego.ai_set_mode('span')
ego.ai_drive_in_lane(True)

## Traffic

To enable traffic, we first spawn a set of vehicles that will be in the traffic pool, and then use the `start_traffic` function. With the traffic, we are no longer alone in this simulation!

#### Documentation

In [39]:
?BeamNGpy.get_available_vehicles

Signature: BeamNGpy.get_available_vehicles(self)
Docstring:
Retrieves a dictionary of vehicles known to the simulator that map
to various properties of the vehicle and a list of pre-configured
vehicle configurations.

Returns:
    A mapping of model names to vehicle properties & configs.

Raises:
    BNGError: If the game is not running to accept a request.
File:      beamngpy\src\beamngpy\beamng.py
Type:      function


In [40]:
?BeamNGpy.start_traffic

Signature: BeamNGpy.start_traffic(self, participants)
Docstring:
Enables traffic simulation for the given list of vehicles.

Args:
    participants (list): List of vehicles that will be part of the
                         simulation. These vehicles need to be spawned
                         beforehand and the simulation will take
                         control of them.
File:      beamngpy\src\beamngpy\beamng.py
Type:      function


In [41]:
?BeamNGpy.switch_vehicle

Signature: BeamNGpy.switch_vehicle(self, vehicle)
Docstring:
Switches to the given :class:`.Vehicle`. This means that the
simulator's main camera, inputs by the user, and so on will all focus
on that vehicle from now on.

Args:
    vehicle (:class:`.Vehicle`): The target vehicle.
File:      beamngpy\src\beamngpy\beamng.py
Type:      function


#### Code

In [42]:
print('The available vehicles to spawn are:', list(beamng.get_available_vehicles()['vehicles'].keys()))

The available vehicles to spawn are: ['flipramp', 'tv', 'tube', 'prosav', 'racetruck', 'large_hamster_wheel', 'autobello', 'large_cannon', 'tirewall', 'trafficbarrel', 'trackedvehicle', 'torsionreactor', 'tsfb', 'moonhawk', 'kickplate', 'inflated_mat', 'idealcar3', 'idealcar2', 'barstow', 'tirestacks', 'shipping_container', 'christmas_tree', 'chair', 'carlift', 'cargotrailer', 'dryvan', 'trashbin', 'vivace', 'car_crusher', 'humvee', 'bollard', 'box', 'frameless_dump', 'suspensionbridge', 'woodplanks', 'large_crusher', 'covet', 'dynamic_col_tester', 'pigeon', 'woodcrate', 'atv', 'idealcar', 'wigeon', 'large_tilt', 'bluebuck', 'streetlight', 'strap', 'generic_traffic', 'miramar', 'coltester', 'large_spinner', 'tiretester_2', 'cardboard_box', 'testroller', 'cones', 'wall', 'bathtub', 'crawler', 'caravan', 'bastion', 'large_bridge', 'shifter_test', 'sunburst', 'midtruck', 'semi', 'piano', 'pickup', 'wendover', 'couch', 'fullsize', 'blockwall', 'hopper', 'openwheel', 'nb_test_rack', 'weight

In [43]:
traffic_vehicles = [
    Vehicle('traffic1', model='miramar'),
    Vehicle('traffic2', model='moonhawk'),
    Vehicle('traffic3', model='sbr'),
    Vehicle('traffic4', model='etki'),
]
for vehicle in traffic_vehicles:
    scenario.add_vehicle(vehicle, (244.87, -926.57, 251.89), rot_quat=(0, 0, 0, 1))
    
beamng.start_traffic(traffic_vehicles)
ego.teleport((-692.93, -1341.58, 140.60), rot_quat=(0.0000, 0.0000, 0.6757, 0.7372))
beamng.switch_vehicle(ego)

## Sensors

Vehicles can be provided with sensors, which capture the simulation and vehicle state data.
The supported sensors include, but are not limited to:
- Camera
- Color camera
- Depth camera
- Semantic annotations
- Instance annotations
- LiDAR
- Inertial measurement units
- Ultrasonic Distance Measurements
- Electrics
- Vehicle state

Let's see how we can use these sensors in BeamNGpy. We respawn our vehicle with multiple sensors and get data from them.

#### Documentation

In [44]:
?Scenario.remove_vehicle

Signature: Scenario.remove_vehicle(self, vehicle)
Docstring:
Removes the given :class:`.Vehicle`: from this scenario. If the
scenario is currently loaded, the vehicle will be despawned.

Args:
    vehicle (:class:`.Vehicle`): The vehicle to remove.
File:      beamngpy\src\beamngpy\scenario.py
Type:      function


In [45]:
?Vehicle.attach_sensor

Signature: Vehicle.attach_sensor(self, name, sensor)
Docstring:
Enters a sensor into this vehicle's map of known sensors and calls the
attach-hook of said sensor. The sensor is identified using the given
name, which has to be unique among the other sensors of the vehicle.

Args:
    name (str): The name of the sensor.
    sensor (:class:`beamngpy.Sensor`): The sensor to attach to the
                                       vehicle.
File:      beamngpy\src\beamngpy\vehicle.py
Type:      function


In [46]:
?Vehicle.poll_sensors

Signature: Vehicle.poll_sensors(self)
Docstring:
Updates the vehicle's sensor readings.

Args:
    requests (None): This function parameter is not used and will be
                     removed in future versions.

Returns:
    Dict with sensor data to support compatibility with
    previous versions.
    Use `vehicle.sensors[<sensor_id>].data[<data_access_id>]` to
    access the polled sensor data.
File:      beamngpy\src\beamngpy\vehicle.py
Type:      function


#### Code

In [47]:
scenario.remove_vehicle(ego)
ego = Vehicle('ego', model='etk800', color='White', licence='PYTHON')
camera = Camera((-0.3, 1, 1), (0, 1, 0), 75, (1024, 1024), colour=True,
                depth=True, annotation=True, instance=True)

ego.attach_sensor('camera', camera)
scenario.add_vehicle(ego, pos=(-692.93, -1341.58, 140.60),
                     rot_quat=(0.0000, 0.0000, 0.6757, 0.7372))
ego.ai_set_mode('span')
ego.ai_drive_in_lane(True)

In [48]:
ego.poll_sensors()
ego.sensors

{'state': <beamngpy.sensors.State at 0x19affdf2640>,
 'camera': <beamngpy.sensors.Camera at 0x19a817baf10>}

### Camera sensor

Camera sensor provides visual information about the environment. Colour camera provides the same visuals as the simulation. Annotation camera provides object annotations - every object class is annotated with a different color. Instance camera provides instance annotation - every object instance is annotated with a different color. And the depth camera provides depth information of the environment.

#### Documentation

In [49]:
?Camera

Init signature:
Camera(
    pos,
    direction,
    fov,
    resolution,
    near_far=(0.01, 1000),
    colour=False,
    depth=False,
    depth_distance=(0.01, 1000),
    depth_inverse=False,
    annotation=False,
    instance=False,
    shmem=True,
)
Docstring:     
A camera sensor provides several types of image data from a user-defined
perspective relative to the vehicle. It can provide the following types of
data:

* Colour images
* Pixel-wise depth
* Pixel-wise object annotation
* Pixel-wise instance annotation to separate overlapping objects of the
  same type

A single camera sensor can be configured to provide any or all of these
data at once, ensuring they all align to the same perspective.
Init docstring:
The camera sensor is set up with a fixed offset position and
directional vector to face relative to the vehicle. This means as the
vehicle moves and rotates, the camera is moved and rotated accordingly.

Besides position and orientation, the image can further be customised


#### Code

In [50]:
ego.sensors['camera'].data

{'type': 'Camera',
 'colour': <PIL.Image.Image image mode=RGBA size=1024x1024 at 0x19AFFF6DFD0>,
 'annotation': <PIL.Image.Image image mode=RGBA size=1024x1024 at 0x19A81AEE640>,
 'depth': <PIL.Image.Image image mode=L size=1024x1024 at 0x19A81AEE7F0>,
 'instance': <PIL.Image.Image image mode=RGBA size=1024x1024 at 0x19A81AEE340>}

In [51]:
fig, ax = plt.subplots(2, 2, figsize=(10, 10))
[axi.set_axis_off() for axi in ax.ravel()]
ax[0, 0].title.set_text('Camera')
ax[0, 1].title.set_text('Depth')
ax[1, 0].title.set_text('Class Annotation')
ax[1, 1].title.set_text('Instance Annotation')

for _ in range(10):
    ego.poll_sensors()
    data = ego.sensors['camera']
    
    ax[0, 0].imshow(np.asarray(data['colour'].convert('RGB')))
    ax[0, 1].imshow(np.asarray(data['depth'].convert('RGB')))
    ax[1, 0].imshow(np.asarray(data['annotation'].convert('RGB')))
    ax[1, 1].imshow(np.asarray(data['instance'].convert('RGB')))
    display(fig)
    clear_output(wait=True)

<Figure size 720x720 with 4 Axes>

### LiDAR

Another sensor provided by the simulation is LiDAR. The LiDAR sensor provides 3D point clouds representing the environment as detected by a pulsing laser emitted from the vehicle.

#### Documentation

In [52]:
?Lidar

Init signature:
Lidar(
    useSharedMemory=False,
    pos=(0, 0, 1.7),
    dir=(0, -1, 0),
    vres=64,
    vAngle=26.9,
    rps=2200000,
    hz=20,
    hAngle=360,
    maxDist=120,
    isVisualised=True,
    isAnnotated=False,
    isStatic=False,
    isSnappingDesired=False,
    isForceInsideTriangle=False,
)
Docstring:      Sensor meta-class declaring methods common to them.
Init docstring:
The LiDAR sensor provides 3D point clouds representing the environment
as detected by a pulsing laser emitted from the vehicle. The range,
position, and refresh rate of this sensor can be customised.
File:           beamngpy\src\beamngpy\sensors.py
Type:           ABCMeta
Subclasses:     


#### Code

In [53]:
scenario.remove_vehicle(ego)
ego = Vehicle('ego', model='etk800', color='White', licence='PYTHON')
lidar = Lidar(useSharedMemory=True)

ego.attach_sensor('lidar', lidar)

scenario.add_vehicle(ego, pos=(-692.93, -1341.58, 140.60),
                     rot_quat=(0.0000, 0.0000, 0.6757, 0.7372))
ego.ai_set_mode('span')

In [54]:
ego.poll_sensors()
ego.sensors['lidar'].data

{'type': 'Lidar',
 'points': array([ -704.67426, -1350.5946 ,   145.59198, ...,     0.     ,
            0.     ,     0.     ], dtype=float32)}

### Ultrasonic sensor

An ultrasonic sensor measures the distance to a close object using ultrasonic waves and uses the echo-localization principle (for example, a parking sensor).

#### Documentation

In [56]:
?Ultrasonic

Init signature:
Ultrasonic(
    pos=(0, -3, 0),
    dir=(0, -1, 0),
    size=(200, 200),
    fov=(0.15, 0.15),
    near_far_planes=(0.05, 10.0),
    range_roundness=-1.15,
    range_cutoff_sensitivity=0.0,
    range_shape=0.3,
    range_focus=0.376,
    range_min_cutoff=0.1,
    range_direct_max_cutoff=5.0,
    sensitivity=3.0,
    fixed_window_size=10.0,
    isVisualised=True,
    isStatic=False,
    isSnappingDesired=False,
    isForceInsideTriangle=False,
)
Docstring:      An ultrasonic sensor (eg parking sensor).
File:           beamngpy\src\beamngpy\sensors.py
Type:           ABCMeta
Subclasses:     


#### Code

In [57]:
scenario.remove_vehicle(ego)
ego = Vehicle('ego', model='etk800', color='White', licence='PYTHON')
ultrasonic = Ultrasonic(isSnappingDesired=True, isForceInsideTriangle=True)

ego.attach_sensor('ultrasonic', ultrasonic)
scenario.add_vehicle(ego, pos=(1079.41, -690.81, 146.81),
                     rot_quat=(-0.0350, -0.0021, -0.7029, 0.7104))

In [58]:
ego.control(gear=1, throttle=0.1)

for _ in range(20):
    ego.poll_sensors()
    print('Distance to obstacle:', ego.sensors['ultrasonic']['distance'])
    beamng.step(30)

Distance to obstacle: 9999.900390625
Distance to obstacle: 9999.900390625
Distance to obstacle: 9999.900390625
Distance to obstacle: 9999.900390625
Distance to obstacle: 9999.900390625
Distance to obstacle: 9999.900390625
Distance to obstacle: 4.292351245880127
Distance to obstacle: 0.614884614944458
Distance to obstacle: 1.0195298194885254
Distance to obstacle: 2.281942844390869
Distance to obstacle: 2.37774395942688
Distance to obstacle: 2.3624916076660156
Distance to obstacle: 2.259110450744629
Distance to obstacle: 2.1889488697052
Distance to obstacle: 2.170274257659912
Distance to obstacle: 2.1712589263916016
Distance to obstacle: 2.1706440448760986
Distance to obstacle: 2.17028546333313
Distance to obstacle: 2.1702685356140137
Distance to obstacle: 2.1702685356140137


### Other sensors

These provide differents kinds of measurements of the vehicle and simulation state:
- `State`: position, direction and velocity of the vehicle
- `IMU`: inertial measurement unit, measures forces and rotational acceleration in the vehicle
- `Electrics`: various values provided by the car's electric systems
- `Damage`: provided exact data about damage of the vehicle parts in the simulation
- `Timer`: provides the relative time since the start of the simulation

#### Documentation

In [59]:
?State

Init signature: State()
Docstring:     
The state sensor monitors general stats of the vehicle, such as position,
direction, velocity, etc. It is a default sensor every vehicle has and is
used to update the vehicle.state attribute.
File:           beamngpy\src\beamngpy\sensors.py
Type:           ABCMeta
Subclasses:     


In [60]:
?IMU

Init signature: IMU(pos=None, node=None, name=None, debug=False)
Docstring:     
An IMU measures forces and rotational acceleration at a certain point on a
vehicle. This can be used to analyze forces acting on certain areas of the
car (like the driver's position) or estimate the trajectory of a vehicle
from its rotation and acceleration.
File:           beamngpy\src\beamngpy\sensors.py
Type:           ABCMeta
Subclasses:     


In [61]:
?Electrics

Init signature: Electrics()
Docstring:     
This sensor is used to retrieve various values made available by the car's
eletrics systems. These values include:

# TODO: List all the electrics.lua values.
- abs (int): ABS state
- abs_active (bool):
- airspeed (float): Airspeed
- airflowspeed (float):
- altitude (float): Z axis position
- avg_wheel_av (float):
- brake (int): Brake value
- brake_lights (int):
- brake_input (int): Brake input value
- check_engine (bool): Check engine light state.
- clutch (int): Clutch value
- clutch_input (int): Clutch input value
- clutch_ratio (int):
- driveshaft (float): Driveshaft
- engine_load (float):
- engine_throttle (int): Engine throttle state
- esc (int): ESC state. 0 = not present/inactive, 1 = disabled, Blink = active
- esc_active (bool):
- exhaust_flow (float):
- fog_lights (int): Fog light state
- fuel (float): Percentage of fuel remaining.
- fuel_capacity (int): Total Fuel Capacity [L].
- fuel_volume (float):
- gear (int):
- gear_a (int): G

In [62]:
?Damage

Init signature: Damage()
Docstring:     
The damage sensor retrieves information about how damaged the structure
of the vehicle is. It's important to realise that this is a sensor that has
no analogue in real life as it returns a perfect knowledge overview of how
deformed the vehicle is. It's therefore more of a ground truth than
simulated sensor data.
File:           beamngpy\src\beamngpy\sensors.py
Type:           ABCMeta
Subclasses:     


In [63]:
?Timer

Init signature: Timer()
Docstring:     
The timer sensor keeps track of the time that has passed since the
simulation started. It provides that information in seconds relative to the
scenario start and does not represent something like a day time or date. It
properly handles pausing the simulation, meaning the value of the timer
sensor does not progress while the simulation is paused.

When polled, this sensor provides the time in seconds since the start of
the scenario in a dictionary under the 'time' key.
File:           beamngpy\src\beamngpy\sensors.py
Type:           ABCMeta
Subclasses:     


#### Code

In [64]:
scenario.remove_vehicle(ego)
ego = Vehicle('ego', model='etk800', color='White', licence='PYTHON')
imu = IMU(pos=(0.73, 0.51, 0.8), debug=True)
electrics = Electrics()
damage = Damage()
timer = Timer()

ego.attach_sensor('imu', imu)
ego.attach_sensor('electrics', electrics)
ego.attach_sensor('damage', damage)
ego.attach_sensor('timer', timer)

scenario.add_vehicle(ego, pos=(-692.93, -1341.58, 140.60),
                     rot_quat=(0.0000, 0.0000, 0.6757, 0.7372))
ego.control(gear=1)
ego.ai_set_mode('span')

In [65]:
ego.poll_sensors()

#### State

In [66]:
ego.sensors['state'].data

{'vel': [1.0375207662582397, 7.189366817474365, 0.003964520525187254],
 'front': [-688.65576171875, -1332.208740234375, 141.10971069335938],
 'rotation': [0.006206502206623554,
  -0.004107812885195017,
  0.9935194253921509,
  -0.11341871321201324],
 'dir': [0.2120097577571869, 0.9772419929504395, 0.007080652751028538],
 'up': [0.01079463493078947, -0.009548216126859188, 0.9998961091041565],
 'pos': [-688.6597900390625, -1333.7484130859375, 140.50440979003906]}

#### IMU

In [67]:
ego.sensors['imu'].data

{'gZ': 49.19240939111148,
 'name': '110194499925',
 'aX': -0.4330600483561859,
 'aY': 0.00045726784263088485,
 'aZ': 1.073448948533815e-16,
 'gX': -4.771048692992466,
 'gY': -6.027084049463459}

#### Electrics

In [68]:
ego.sensors['electrics'].data

{'hasESC': 1,
 'gearboxMode': 'arcade',
 'steeringUnassisted': -0.24448073453469935,
 'wheelThermals': {'FR': {'brakeCoreTemperature': 25.936757637423113,
   'brakeSurfaceTemperature': 25.936730957031273,
   'brakeThermalEfficiency': 0.9309739557990574},
  'FL': {'brakeCoreTemperature': 25.936757639399545,
   'brakeSurfaceTemperature': 25.936730957031273,
   'brakeThermalEfficiency': 0.9309739557990574},
  'RR': {'brakeCoreTemperature': 25.936755207115013,
   'brakeSurfaceTemperature': 25.936730957031273,
   'brakeThermalEfficiency': 0.9309739557990574},
  'RL': {'brakeCoreTemperature': 25.93916548314607,
   'brakeSurfaceTemperature': 25.936730957031273,
   'brakeThermalEfficiency': 0.9309739557990574}},
 'nop': 0,
 'maxrpm': 6800,
 'minGearIndex': -1,
 'maxGearIndex': 6,
 'reverse': 0,
 'rpmspin': 183.3904863079467,
 'boost': 0,
 'boostMax': 0,
 'lights': 0,
 'turnsignal': 0,
 'altitude': 140.50363823922922,
 'abs': 0,
 'dseColor': '238BE6',
 'parkingbrake': 0,
 'lightbar': 0,
 'airsp

#### Damage

In [69]:
ego.sensors['damage'].data

{'damage': 0,
 'damage_ext': 0,
 'type': 'Damage',
 'part_damage': [],
 'deform_group_damage': {'wheelaxleRL': {'eventCount': 0,
   'maxEvents': 100,
   'invMaxEvents': 0.01,
   'damage': 0},
  'fendersignal_L_break': {'eventCount': 0,
   'maxEvents': 1299.9999999999998,
   'invMaxEvents': 0.0007692307692307693,
   'damage': 0},
  'wheelaxleRR': {'eventCount': 0,
   'maxEvents': 100,
   'invMaxEvents': 0.01,
   'damage': 0},
  'radtube_break': {'eventCount': 0,
   'maxEvents': 499.99999999999994,
   'invMaxEvents': 0.002,
   'damage': 0},
  'doorglass_RR_break': {'eventCount': 0,
   'maxEvents': 639.9999999999999,
   'invMaxEvents': 0.0015625000000000003,
   'damage': 0},
  'windshield_break': {'eventCount': 0,
   'maxEvents': 5249.999999999999,
   'invMaxEvents': 0.0001904761904761905,
   'damage': 0},
  'doorglass_FL_break': {'eventCount': 0,
   'maxEvents': 659.9999999999999,
   'invMaxEvents': 0.0015151515151515154,
   'damage': 0},
  'foglightglass_R_break': {'eventCount': 0,
   '

#### Timer

In [70]:
ego.sensors['timer'].data

{'time': 171.73650391586125}

### Where next

And this is the end of the quick overview of BeamNGpy features! If you want to see more examples, please follow the [example guide](https://github.com/BeamNG/BeamNGpy/blob/dev/examples/guide.md).